# GitHub repositories analysis

In [2]:
import pandas as pd

## Read the data

In [3]:
df = pd.read_csv("data.csv", index_col=0)

In [4]:
df.head(3)

,name,ts,stars_count,views_total,views_uniques,clones_total,clones_uniques
amq-streams-workshop-2022/06/28,amq-streams-workshop,2022/06/28,0,7,1,0,0
b0rr3g0-blog-2022/07/07,b0rr3g0-blog,2022/07/07,0,15,1,5,4
b0rr3g0-blog-2022/07/08,b0rr3g0-blog,2022/07/08,0,6,1,9,5


## Data analysis

### Views

In [5]:
views = df.groupby(['name']).agg({"views_total": ["sum"]})
views = views[views[('views_total', 'sum')] > 0]
views.sort_values(by=[('views_total', 'sum')], ascending=False)

,views_total
,sum
name,
kotlin-ktor,159
golang-rest-api-cassandra,88
spring-kotlin-crud,54
quarkus-kotlin-example,39
ta,38
pelorus-workshop,31
iot-esp8266-irrigation-system,30
library,28


### Clones

In [6]:
clones = df.groupby(['name']).agg({"clones_uniques": ["sum"]})
clones = clones[clones[('clones_uniques', 'sum')] > 0]
clones.sort_values(by=[('clones_uniques', 'sum')], ascending=False)

,clones_uniques
,sum
name,
ta,251
golang-rest-api-cassandra,190
mqtt-golang-influxdb,100
iot-devices-crud,87
pelorus-workshop,14
ml-learning,11
b0rr3g0-blog,9
service-mesh-wasm-go,4


# Stars

In [9]:
stars = df.groupby(['name']).agg({"stars_count": ["max"]})
stars = stars[stars[('stars_count', 'max')] > 0]
stars.sort_values(by=[('stars_count', 'max')], ascending=False)

,stars_count
,max
name,
mqtt-golang-influxdb,9
spring-kotlin-crud,6
kotlin-ktor,5
kops-aws,4
kustomize-vs-helm,3
ta,3
golang-k8s-helm-helloworld,2
golang-rest-api-cassandra,2
